In [ ]:
""" 
Author: p.wagner@bhvi.org / p.wagner@unsw.edu.au 
image registration of anterior segment projection from several scans 

Purpose: 
use averaged rnfl data [per participant, eye, scan_type] to identify macula positioning 

use inter participant adjustment for macula positioning to match choroid thickness maps per and post intervention 

calculate final residual 


"""
import pandas as pd
import numpy as np
import os.path
from pathlib import Path
from pystackreg import StackReg
from skimage import io
from scipy import stats

import sys
sys.path.append(r'C:\Users\p.wagner\Documents\Python Scripts\oct_data_analyses_helpers')
from oct_helpers_lib import OctDataAccess as get_px_meta
from oct_helpers_lib import TopconSegmentationData as TSD

import cv2
import glob
from PIL import Image
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px

path_oct = r'E:\studyIII\OCT_data'
path_logbook = r'C:\Users\p.wagner\Documents\phd\C4 study_III\participants'
fn_logbook = 'participant_log_studyIII_V0.2.xlsx' 
fp_fn_logbook = os.path.join(path_logbook, fn_logbook)
thickness_csv = 'DEFAULT_3D_All_Thickness.csv'


# check if path_oct is available 
if not os.path.isdir(path_oct):
    print('OCT data path NOT available')

In [ ]:
def create_rnfl0_pos_im(px_id, csv_fn, fp_oct, rnfl_0_idx):
    fig_output_fn = (str(px_id) + '_' + fn.split('\\')[-1].split('.')[0] + '_rnfl_0_thickness_macula.png')

    # scatter plot of positions 
    plt.scatter(rnfl_0_idx[0], rnfl_0_idx[1], s=1)
    plt.scatter(np.mean(rnfl_0_idx[0]), np.mean(rnfl_0_idx[1]), s=20, c='red')
    plt.gca().set_aspect('equal')
    plt.xlim([175, 325])
    plt.ylim([50, 200])
    plt.title = ('PX_id: ' + str(px_id) + ', Scan set: ' + fn.split('\\')[-1].split('.')[0]) 

    
    plt.savefig(os.path.join(fp_oct, 'images\\rnfl0', fig_output_fn))
    plt.clf()



In [ ]:
def average_with_referece_to_macula(macula_pos_all, scan_type, layer_id, incl_std_factor=2):
    # calculate OD initial scan average with adjustment for macula positioning
    
    slice_width = 16
    nan_counter = []
    
    eye_id = scan_type.split('_')[2]
    scan_set = scan_type.split('_')[1]
    macula_pos_set = macula_pos_all.loc[(macula_pos_all.eye_id == eye_id) &
                                     (macula_pos_all.scan_type == scan_set), :].reset_index(drop=True)
    layer_fn_pre = '_'.join([eye_id, 'initial', layer_id, 'mean_regV02.csv'])
    layer_fn_post= '_'.join([eye_id, 'post', layer_id, 'mean_regV02.csv'])
    
    layer_all = []
    for idx, layer_fp in enumerate(macula_pos_set.fp):
        pos_adjust_layer = np.empty((256, 512), dtype=float)
        pos_adjust_layer[:] = np.nan
        
        layer_initial = pd.read_csv(os.path.join(layer_fp, layer_fn_pre), index_col=0).values.astype(float) 
        layer_post = pd.read_csv(os.path.join(layer_fp, layer_fn_post ), index_col=0) .values.astype(float)
        layer_all_avg = np.mean(np.array([np.array(layer_post), np.array(layer_initial)]), axis=0)

        if scan_set == 'initial':
            layer = np.array(layer_initial - layer_all_avg)
        if scan_set == 'post':
            layer = np.array(layer_post - layer_all_avg )
                             
        row_pos = int(macula_pos_set.loc[idx, 'row_dev'])
        col_pos = int(macula_pos_set.loc[idx, 'col_dev'])

        print(idx, ' ', macula_pos_set.loc[idx, 'row_dev'], ' ', macula_pos_set.loc[idx, 'col_dev'])
        if (row_pos <= 0) & (col_pos <= 0):
            pos_adjust_layer[- row_pos: 256, - col_pos:512] = layer[0:256 + row_pos, 0:512 + col_pos]
        #         print ('< <')
        elif (row_pos <= 0) & (col_pos >= 0):
            pos_adjust_layer[- row_pos: 256, 0:512 - col_pos] = layer[0:256 + row_pos, col_pos:512]
        #         print( '< > ')
        elif (row_pos >= 0) & (col_pos <= 0):
            pos_adjust_layer[0: 256 - row_pos, - col_pos:512] = layer[row_pos:256, 0: 512 + col_pos]
        #         print( '> < ')
        elif (row_pos >= 0) & (col_pos >= 0):
            pos_adjust_layer[0:256 - row_pos, 0:512 - col_pos] = layer[row_pos: 256, col_pos: 512]
        #         print( '> > ')
        #     else:
        #         pos_adjust_rnfl = rnfl
        #         print( '= =')

        # set filter for invalid data exclusion 
        mean_c_map = np.nanmean(pos_adjust_layer)
        std_c_map = np.nanstd(pos_adjust_layer)
        print(mean_c_map, std_c_map)
        pos_adjust_layer[pos_adjust_layer > ( incl_std_factor * std_c_map + mean_c_map)] = np.nan
        pos_adjust_layer[pos_adjust_layer < (-incl_std_factor * std_c_map + mean_c_map)] = np.nan
        
        
#         for idx, x in enumerate(range(0, 513-slice_width, slice_width)):
#             slice_mean = np.nanmean(pos_adjust_layer[:, x : x+slice_width])
#             slice_std = np.nanstd(pos_adjust_layer[:, x : x+slice_width])
#             pos_adjust_layer[:, x : x+slice_width][np.isnan(pos_adjust_layer[:, x : x+slice_width])] = ((np.random.normal(slice_mean, 
#                                                                                         slice_std, 
#                                                                                         (pos_adjust_layer[:, x : x+slice_width]
#                                                                           [np.isnan(pos_adjust_layer[:, x : x+slice_width])].size))) 
#                                                                       )

        
        layer_all.append(pos_adjust_layer)
        # count all pixel with depth data 
        idxs = np.where(np.isnan(pos_adjust_layer)) 
        mask = np.zeros(pos_adjust_layer.shape)
        mask[idxs[0], idxs[1]] = 1
        nan_counter.append(mask)
        
    layer_all = np.array(layer_all)
    nan_counter = np.nansum(np.array(nan_counter), axis=0)
    return layer_all, nan_counter

In [ ]:

# mean_all = np.nanmean(layer_all, axis=0)
# std_all = np.nanstd(layer_all, axis=0)
# max_all = np.nanmax(layer_all, axis=0)
# min_all = np.nanmin(layer_all, axis=0)

In [ ]:

# choroid_initial = np.array(OD_pre_layers)
# choroid_post = np.array(OD_post_layers)

def appl_stats_on_map(map1, map2):
    p_values = np.empty((256, 512), dtype=float) 

    for row_idx in range(0,255): 
        for col_idx in range(0,511):
            array1 = map1[:, row_idx, col_idx]
            array2 = map2[:, row_idx, col_idx]

            p_values[row_idx, col_idx] = (stats.wilcoxon(array1, array2)[1])
    return p_values

def map_pValues(display_data, save_file=False, output_fpn=None):
    # creating a accumulative map of dioptric landscape
    fig = px.imshow(display_data,
                    title=title_name,
                    labels=dict(x='Horizontal decentration from fovea in degrees [°]',
                                y='Vertical decentration from fovea in degrees [°]',
                                color="p-Values",
                                ),
#                     # pixels to degrees translation
                    x=np.array([float(x) * 0.0703 for x in range(0, 512)]) - 18,
                    y=-1 * np.array([float(x) * 0.10547 for x in range(0, 256)]) + 13.5,
#                         y = 1 * np.array([int(x) for x in range(0, 256)])
                    )
    fig.update_xaxes(side="bottom")
    fig.update_yaxes(autorange=True)

    fig.update_coloraxes(
                          colorscale=[[0.0, "rgb(255,  0,  0)"],
                                     [0.05, "rgb(255,  0,  0)"],
                                     [0.05, "rgb(125,125,  0)"],
                                     [0.1,  "rgb(125,125,  0)"],
                                     [0.1,  "rgb(255,255,255)"],
#                                      [0.2,  "rgb(200,125,100)"],
#                                      [0.2,  "rgb(200,200,100)"],
#                                      [0.3,  "rgb(200,200,100)"],
#                                      [0.3,  "rgb(200,200,200)"],
#                                      [0.5,  "rgb(200,200,200)"],
                                     [1.0,  "rgb(155,155,155)"]],
                        colorbar_title_side='right',
                        colorbar_tickfont=dict(size=18, ),
                        colorbar_title_font=dict(size=18, ),
                        reversescale=False
                         )

    fig.update_layout(title_font_size=24,
                      autosize=True,
                      width=1200, height=900,
                      margin=dict(l=10, r=10, b=10, t=40),
                      )
    fig.update_xaxes(title_font=dict(size=22, ), tickfont=dict(size=18), )
    fig.update_yaxes(title_font=dict(size=22, ), tickfont=dict(size=18), )

    if save_file:
        fig.write_image(output_fpn)
    else:
        fig.show()
        
def map_significant_data(display_data, save_file=False, output_fpn=None):
    # creating a accumulative map of dioptric landscape
    fig = px.imshow(display_data,
                    title=title_name,
                    labels=dict(x='Horizontal decentration from fovea in degrees [°]',
                                y='Vertical decentration from fovea in degrees [°]',
                                color="thickness change in micrometer",
                                ),
#                     # pixels to degrees translation
                    x=np.array([float(x) * 0.0703 for x in range(0, 512)]) - 18,
                    y=-1 * np.array([float(x) * 0.10547 for x in range(0, 256)]) + 13.5,
#                         y = 1 * np.array([int(x) for x in range(0, 256)])
                    )
    fig.update_xaxes(side="bottom")
    fig.update_yaxes(autorange=True)

    fig.update_coloraxes(cmin=-30,
                         cmax= 30,
                         colorscale='Portland',

#                           colorscale=[[0.0, "rgb(255,  0,  0)"],
#                                      [0.05, "rgb(255,  0,  0)"],
#                                      [0.05, "rgb(125,125,  0)"],
#                                      [0.1,  "rgb(125,125,  0)"],
#                                      [0.1,  "rgb(255,255,255)"],
# #                                      [0.2,  "rgb(200,125,100)"],
# #                                      [0.2,  "rgb(200,200,100)"],
# #                                      [0.3,  "rgb(200,200,100)"],
# #                                      [0.3,  "rgb(200,200,200)"],
# #                                      [0.5,  "rgb(200,200,200)"],
#                                      [1.0,  "rgb(155,155,155)"]],
                        colorbar_title_side='right',
                        colorbar_tickfont=dict(size=18, ),
                        colorbar_title_font=dict(size=18, ),
                        reversescale=False,
                         )

    fig.update_layout(title_font_size=24,
                      autosize=True,
                      width=1200, height=900,
                      margin=dict(l=10, r=10, b=10, t=40),
                      )
    fig.update_xaxes(title_font=dict(size=22, ), tickfont=dict(size=18), )
    fig.update_yaxes(title_font=dict(size=22, ), tickfont=dict(size=18), )

    if save_file:
        fig.write_image(output_fpn)
    else:
        fig.show()



In [ ]:
px_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 23, 26, ] # all px
# px_ids = [1,    3, 4, 5, 6,       9, 10, 11,     13, 14, 15, 16, 17, 18, 20, 23, ] # bad scan out  

# px_ids =[16, ]

scan_types_all = ['OCT_initial_OD', 'OCT_post_OD', 'OCT_initial_OS', 'OCT_post_OS']
    

column_names = ['px_id', 'eye_id', 'scan_type', 'fp', 'macula_row_pos', 'macula_col_pos']    
macula_pos_all = pd.DataFrame(columns=column_names)
for px_id in px_ids:
    px_meta_data = get_px_meta(fp_fn_logbook, [px_id,], scan_types_all, path_oct)
    for fn in glob.glob(os.path.join(px_meta_data.subject_rec_fp[0] ,'*rnfl_mean_reg.csv')):
        # get scan type 
        eye_id = fn.split('\\')[-1].split('.')[0].split('_')[0]
        scan_type = fn.split('\\')[-1].split('.')[0].split('_')[1]    
        # read fn 
        rnfl_data = pd.read_csv(fn, index_col=0)
        # look in interval [50:200, 175:325] for 0 pos or smaler xx for macula position 
        rnfl_macula = rnfl_data.iloc[50:200, 175:325].values
        # calculate centre position 
        # horizontal positioning == column space counted from the top left
        row_pos = np.mean(np.where(rnfl_macula < 1)[0] + 50) 
        # vertical positionng == row space 
        col_pos = np.mean(np.where(rnfl_macula < 1)[1] + 175)


        rnfl_macula_idxs = (np.where(rnfl_macula < 1)[1] + 175),  (256 + -(np.where(rnfl_macula < 1)[0] + 50))

        macula_pos_all = macula_pos_all.append(pd.DataFrame({'px_id':px_id,
                                     'eye_id' : eye_id,  
                                     'scan_type': scan_type,
                                     'fp': '\\'.join(fn.split('\\')[:-1]),
                                     'macula_row_pos': [row_pos], 
                                     'macula_col_pos': [col_pos]
                                                              }))
        
#         # create scatter plot and save to file for quality check
#         create_rnfl0_pos_im(px_id, fn, path_oct, rnfl_macula_idxs)
        
#         print(fn)
# record center position for all averged sets of scans                       

In [ ]:
# calculating residual of center positions for each eye across pre and post intervention 
macula_pos_all.loc[macula_pos_all.eye_id =='OD', 'col_dev' ] = np.round(macula_pos_all.loc[macula_pos_all.eye_id =='OD',  'macula_col_pos']-
                                                                np.mean(macula_pos_all.loc[macula_pos_all.eye_id =='OD',  'macula_col_pos']), 0)

macula_pos_all.loc[macula_pos_all.eye_id =='OD', 'row_dev' ] = np.round(macula_pos_all.loc[macula_pos_all.eye_id =='OD',  'macula_row_pos']-
                                                                np.mean(macula_pos_all.loc[macula_pos_all.eye_id =='OD',  'macula_row_pos']), 0)                                                                                   

macula_pos_all.loc[macula_pos_all.eye_id =='OS', 'col_dev' ] = np.round(macula_pos_all.loc[macula_pos_all.eye_id =='OS',  'macula_col_pos']-
                                                                np.mean(macula_pos_all.loc[macula_pos_all.eye_id =='OS',  'macula_col_pos']), 0)

macula_pos_all.loc[macula_pos_all.eye_id =='OS', 'row_dev' ] = np.round(macula_pos_all.loc[macula_pos_all.eye_id =='OS',  'macula_row_pos']-
                                                                np.mean(macula_pos_all.loc[macula_pos_all.eye_id =='OS',  'macula_row_pos']), 0) 
                                                                                                


In [ ]:
# # apply residual to average across averaged rnfl maps 
std_incl_factor = 2
layer_id = 'choroid'
scan_type = 'OCT_initial_OD'

OD_pre_layers, OD_pre_nans = average_with_referece_to_macula(macula_pos_all, scan_type, layer_id, std_incl_factor)

scan_type = 'OCT_post_OD'

OD_post_layers, OD_post_nans = average_with_referece_to_macula(macula_pos_all, scan_type, layer_id, std_incl_factor)

In [ ]:
# output_fp = r'E:\studyIII\OCT_data\images\residuals'

# OD_residual = OD_post_layers - OD_pre_layers
# # OS_residual = OS_pre_layers - OS_post_layers

# for idx, px_id in enumerate(px_ids):
#     output_fn = str(px_id) + '_OD_choroid_thickness_residual_pre_post.png'
#     output_fpn = os.path.join(output_fp, output_fn)
#     display_data = OD_residual[idx, :,:]
    
#     title_name = ('Choroid thickness residual OD, px_id: ' + str(px_id) +
#                   ', avg: ' + str(np.round(np.nanmean(display_data), 2)) +
#                   ', std: ' + str(np.round(np.nanstd(display_data), 2))  +
#                   ', max: ' + str(np.round(np.nanmax(display_data), 2)) + 
#                   ', min: ' + str(np.round(np.nanmin(display_data), 2))        
#                  ) 
#     map_significant_data(display_data, save_file=True, output_fpn=output_fpn)
    

In [ ]:
# # save data to a csv file 
# output_fp = r'E:\studyIII\OCT_final_data\95perc'
# for idx, px_id in enumerate(px_ids):
#     # pre intervention
#     output_fn = '_'.join(['choroid_thickness_map_95perc_initial', 
#                           str(px_id), 
#                           scan_type.split('_')[-1],
#                          '.csv'])
#     output_layer = pd.DataFrame (OD_pre_layers[idx, :,:])
#     output_layer.to_csv(os.path.join(output_fp, output_fn), index=True)
#     # post intervention
#     output_fn = '_'.join(['choroid_thickness_map_95perc_post', 
#                           str(px_id), 
#                           scan_type.split('_')[-1],
#                          '.csv'])
#     output_layer = pd.DataFrame (OD_post_layers[idx, :,:])
#     output_layer.to_csv(os.path.join(output_fp, output_fn), index=True)


In [ ]:
#  create box plot right left and centre in quarters 

max_exclusion = 5
mask = np.ones((256, 512))
idxs = np.where(OD_pre_nans>=max_exclusion)
mask[idxs[0], idxs[1]] = 0
idxs = np.where(OD_post_nans>=max_exclusion)
mask[idxs[0], idxs[1]] = 0
idxs = np.where(mask == 0)

OD_initial_mean = np.nanmean(OD_pre_layers, axis=0)
OD_post_mean = np.nanmean(OD_post_layers, axis=0)

# dispaly_data
display_data = (OD_post_mean - OD_initial_mean) 
display_data[idxs] = np.nan

left = display_data[0:256, 0:128][~np.isnan(display_data[0:256, 0:128])]

centre = display_data[50: 200, 192:320][~np.isnan(display_data[50: 200, 192:320])]

right = display_data[0:256, 384:512][~np.isnan(display_data[0:256, 384:512])]


plt.figure(figsize=(12, 8), dpi=80)
data = [left, centre, right]
fig7, ax7 = plt.subplots(figsize=(12, 8), dpi=80)
ax7.set_title('Effects of optical defocus on central (foveal) and peripheral retina')
ax7.boxplot(data)
plt.xticks([1, 2, 3], ['myopic defocus peripheral left', 'centre fovea area', 'hyperopic defocus peripheral right'])
plt.grid(axis='y')
plt.show()


In [ ]:
# sumarize data from areas and record in table, to csv

scan_types_all = ['OCT_initial_OD', 'OCT_post_OD', 'OCT_initial_OS', 'OCT_post_OS']
column_names = ['px_id', 'scan_type', 'overall_mean', 'overall_std', 
                'lhs_area_mean', 'lhs_area_std',
                'centre_area_mean', 'centre_area_std',
                'rhs_area_mean', 'rhs_area_std',
               ]
df = pd.DataFrame(columns=column_names)
for idx, px_id in enumerate(px_ids):
    for scan_type, data in zip(scan_types_all, [OD_pre_layers, OD_post_layers, OS_pre_layers, OS_post_layers]):
        df_new = pd.DataFrame(columns=column_names)
        df_new.loc[0,'px_id'] = px_id
        df_new.loc[0,'scan_type'] = scan_type
        df_new.loc[0,'overall_mean'] = np.nanmean(data[idx, :,:])
        df_new.loc[0,'overall_std'] =  np.nanstd( data[idx, :,:])                                         

        left = data[idx, 0:256, 0:128][~np.isnan(data[idx, 0:256, 0:128])]
        df_new.loc[0,'lhs_area_mean'] = np.nanmean(left)
        df_new.loc[0,'lhs_area_std'] =  np.nanstd( left)                                         

        centre = data[idx, 50: 200, 192:320][~np.isnan(data[idx, 50: 200, 192:320])]
        df_new.loc[0,'centre_area_mean'] = np.nanmean(centre)
        df_new.loc[0,'centre_area_std'] =  np.nanstd(centre)  
        
        right = data[idx, 0:256, 384:512][~np.isnan(data[idx, 0:256, 384:512])]
        df_new.loc[0,'rhs_area_mean'] = np.nanmean(right)
        df_new.loc[0,'rhs_area_std'] =  np.nanstd(right)  

        df = df.append(df_new)
df.to_csv('mean_residuals_in_different_areas_' + str(std_incl_factor) + 'std_incl.csv')

In [ ]:
lhs_data = df.loc[df.scan_type == 'OCT_post_OD', 'lhs_area_mean'] - df.loc[df.scan_type == 'OCT_initial_OD', 'lhs_area_mean']
centre_data = df.loc[df.scan_type == 'OCT_post_OD', 'centre_area_mean'] - df.loc[df.scan_type == 'OCT_initial_OD', 'centre_area_mean']
rhs_data = df.loc[df.scan_type == 'OCT_post_OD', 'rhs_area_mean'] - df.loc[df.scan_type == 'OCT_initial_OD', 'rhs_area_mean']


plt.figure(figsize=(12, 8), dpi=80)
data = [lhs_data, centre_data, rhs_data]
fig7, ax7 = plt.subplots(figsize=(12, 8), dpi=80)
ax7.set_title('Effects of optical defocus on central (foveal) and peripheral retina')
ax7.boxplot(data)
plt.xticks([1, 2, 3], ['myopic defocus peripheral left', 'centre fovea area', 'hyperopic defocus peripheral right'])
plt.grid(axis='y')
plt.show()


In [ ]:
# OD_pre_layers, OD_post_layers, OS_pre_layers, OS_post_layers
OD_residual = OD_post_layers - OD_pre_layers
OS_residual = OS_post_layers - OS_pre_layers

OD_residual_mean = []
OD_residual_std =[]
OD_left_area_means = []
OD_centre_area_means = []
OD_right_area_means = []

OS_residual_mean = []
OS_residual_std =[]
OS_left_area_means = []
OS_centre_area_means = []
OS_right_area_means = []

for idx, px_id in enumerate(px_ids):
    
    OD_residual_mean.append(np.nanmean(OD_residual[idx, :, :]))
    OD_residual_std.append(np.nanmean(OD_residual[idx, :, :]))  
    OD_left_area_means.append(np.nanmean(OD_residual[idx, 0:256, 0:128]))
    OD_centre_area_means.append(np.nanmean(OD_residual[idx, 50: 200, 192:320]))
    OD_right_area_means.append(np.nanmean(OD_residual[idx, 0:256, 384:512]))
    
    OS_residual_mean.append(np.nanmean(OS_residual[idx, :, :]))
    OS_residual_std.append(np.nanmean(OS_residual[idx, :, :]))     
    OS_left_area_means.append(np.nanmean(OS_residual[idx, 0:256, 0:128]))
    OS_centre_area_means.append(np.nanmean(OS_residual[idx, 50: 200, 192:320]))
    OS_right_area_means.append(np.nanmean(OS_residual[idx, 0:256, 384:512]))
    
plt.figure(figsize=(12, 8), dpi=80)
data = [OD_left_area_means, OD_centre_area_means, OD_right_area_means,
       OS_left_area_means, OS_centre_area_means, OS_right_area_means,
       ]
fig7, ax7 = plt.subplots(figsize=(12, 8), dpi=80)
ax7.set_title('Effects of optical defocus on central (foveal) and peripheral retina')
ax7.boxplot(data)
plt.xticks([1, 2, 3], ['myopic defocus peripheral left', 'centre fovea area', 'hyperopic defocus peripheral right'])
plt.grid(axis='y')
plt.show()


In [ ]:
for idx, px_id in enumerate(px_ids):
    print(px_id, 'OD residual mean: ', np.round(np.nanmean(OD_residual[idx, :,:]), 2), 
         'std: ', np.round(np.nanstd(OD_residual[idx, :,:]), 2))

In [ ]:
OD_residual.shape

In [ ]:
# OD_pre_nans[OD_pre_nans>10] = 10

# display_data = OD_pre_nans
# display_data[display_data>10] = 10
fig = go.Figure(data=[go.Surface(z=-display_data)])


fig.update_layout(scene_camera_eye=dict(y=-4, x=0.01, z=-12),
                  scene_aspectmode='manual', 
                  scene_aspectratio=dict(x=12, y=9, z=10),
                  scene = dict(xaxis = dict(nticks=10, range=[0,512],),
                               yaxis = dict(nticks=10, range=[0,256],),
                               zaxis = dict(nticks=10, range=[-30,30],),),)

fig.show()

In [ ]:
max_exclusion = 10
mask = np.ones((256, 512))
idxs = np.where(OD_pre_nans>=max_exclusion)
mask[idxs[0], idxs[1]] = 0
idxs = np.where(OD_post_nans>=max_exclusion)
mask[idxs[0], idxs[1]] = 0
idxs = np.where(mask == 0)

OD_initial_mean = np.nanmean(OD_pre_layers, axis=0)
OD_post_mean = np.nanmean(OD_post_layers, axis=0)

OD_initial_std = np.nanstd(OD_pre_layers, axis=0)
OD_post_std = np.nanstd(OD_post_layers, axis=0)

display_data = (OD_post_mean - OD_initial_mean) 
display_data[idxs] = np.nan

fig = go.Figure(data=[go.Surface(z=np.array(display_data.T))])

title_name = ('Choroid thickness, '+ scan_type.split('_')[-1] +', STD post intervention' +
              ', avg: ' + str(np.round(np.nanmean(display_data), 2)) +
              ', std: ' + str(np.round(np.nanstd(display_data), 2))  +
              ', max: ' + str(np.round(np.nanmax(display_data), 2)) + 
              ', min: ' + str(np.round(np.nanmin(display_data), 2)) 
             ) 

fig.update_layout(title=title_name, title_font_size=22, autosize=False,
                  width=1200, height=900,
                  margin=dict(l=50, r=50, b=50, t=50), 
                  )

fig.update_layout(scene_camera_eye=dict(x=4, y=-0.01, z=22),
                  scene_aspectmode='manual', 
                  scene_aspectratio=dict(x=9, y=12, z=10),
                  scene = dict(xaxis = dict(nticks=10, range=[0,256],),
                               yaxis = dict(nticks=10, range=[0,512],),
                               zaxis = dict(nticks=10, range=[-30,30],),),)
fig.update_layout(scene = dict(
                    xaxis_title='vertical pixel position',
                    yaxis_title='thickness map, horizontal pixel position',
                    zaxis_title='thickness in micrometer'),
                 )

fig.show()


In [ ]:
print('mean: ', np.nanmean(display_data), ' std: ', np.nanstd(display_data))
fig_hist =plt.hist(display_data.flatten(), 100)

map_significant_data(display_data, save_file=False, output_fpn=None)


In [ ]:
# calculating p-Values with Wilcoxon stats test
p_values = appl_stats_on_map(OD_post_layers, OD_pre_layers)

In [ ]:
mask = np.ones((256, 512))
idxs = np.where(OD_pre_nans>=max_exclusion)
mask[idxs[0], idxs[1]] = 0
idxs = np.where(OD_post_nans>=max_exclusion)
mask[idxs[0], idxs[1]] = 0
idxs = np.where(mask == 0)


display_data = p_values
display_data[idxs[0], idxs[1]] = np.nan

im_output_fn = 'OD_all_wilcoxon_choroid_thickness_delta_reg_mm_outlierremoval.png'

title_name = ('Choroid thickness mean residual stats, OD, all px, Wilcoxon signed-rank test') 
map_pValues(p_values)


In [ ]:
display_data = (OD_post_mean - OD_initial_mean) 

idxs = np.where(OD_pre_nans>=max_exclusion)
display_data[idxs] = np.nan
idxs = np.where(OD_post_nans>=max_exclusion)
display_data[idxs] = np.nan

idxs = np.where(p_values>0.05)
display_data[idxs] = np.nan
title_name = ('Choroid thickness areas of significant change, OD, stats Wilcoxon signed-rank test') 
map_significant_data(display_data, save_file=False, output_fpn=None)

In [ ]:
display_data = OD_pre_nans
title_name = ('count of valid data form participants per pixel position') 
map_significant_data(display_data, save_file=False, output_fpn=None)


In [ ]:
# # # OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS

In [ ]:
# apply residual to average across averaged rnfl maps 
std_incl_factor = 2
layer_id = 'choroid'
scan_type = 'OCT_initial_OS'

OS_pre_layers, OS_pre_nans = average_with_referece_to_macula(macula_pos_all, scan_type, layer_id, std_incl_factor)

layer_id = 'choroid'
scan_type = 'OCT_post_OS'

OS_post_layers, OS_post_nans = average_with_referece_to_macula(macula_pos_all, scan_type, layer_id, std_incl_factor)

In [ ]:
# output_fp = r'E:\studyIII\OCT_data\images\residuals'

# # OD_residual = OD_pre_layers - OD_post_layers
# OS_residual = OS_post_layers - OS_pre_layers

# for idx, px_id in enumerate(px_ids):
#     output_fn = str(px_id) + '_OS_choroid_thickness_residual_pre_post.png'
#     output_fpn = os.path.join(output_fp, output_fn)
#     display_data = OS_residual[idx, :,:]
    
#     title_name = ('Choroid thickness residual OS, px_id: ' + str(px_id) +
#                   ', avg: ' + str(np.round(np.nanmean(display_data), 2)) +
#                   ', std: ' + str(np.round(np.nanstd(display_data), 2))  +
#                   ', max: ' + str(np.round(np.nanmax(display_data), 2)) + 
#                   ', min: ' + str(np.round(np.nanmin(display_data), 2))        
#                  ) 
#     map_significant_data(display_data, save_file=True, output_fpn=output_fpn)

In [ ]:
# # save data to a csv file 
# output_fp = r'E:\studyIII\OCT_final_data\95perc'
# for idx, px_id in enumerate(px_ids):
#     # pre intervention
#     output_fn = '_'.join(['choroid_thickness_map_95perc_initial', 
#                           str(px_id), 
#                           scan_type.split('_')[-1],
#                          '.csv'])
#     output_layer = pd.DataFrame (OS_pre_layers[idx, :,:])
#     output_layer.to_csv(os.path.join(output_fp, output_fn), index=True)
#     # post intervention
#     output_fn = '_'.join(['choroid_thickness_map_95perc_post', 
#                           str(px_id), 
#                           scan_type.split('_')[-1],
#                          '.csv'])
#     output_layer = pd.DataFrame (OS_post_layers[idx, :,:])
#     output_layer.to_csv(os.path.join(output_fp, output_fn), index=True)

In [ ]:
#  create box plot right left and centre in quarters 
#  create box plot right left and centre in quarters 

max_exclusion = 5
mask = np.ones((256, 512))
idxs = np.where(OS_pre_nans>=max_exclusion)
mask[idxs[0], idxs[1]] = 0
idxs = np.where(OS_post_nans>=max_exclusion)
mask[idxs[0], idxs[1]] = 0
idxs = np.where(mask == 0)

OS_initial_mean = np.nanmean(OS_pre_layers, axis=0)
OS_post_mean = np.nanmean(OS_post_layers, axis=0)

# dispaly_data
display_data = (OS_post_mean - OS_initial_mean) 
display_data[idxs] = np.nan

left = display_data[0:256, 0:128][~np.isnan(display_data[0:256, 0:128])]

centre = display_data[50: 200, 192:320][~np.isnan(display_data[50: 200, 192:320])]

right = display_data[0:256, 384:512][~np.isnan(display_data[0:256, 384:512])]


plt.figure(figsize=(12, 8), dpi=80)
data = [left, centre, right]
fig7, ax7 = plt.subplots(figsize=(12, 8), dpi=80)
ax7.set_title('Effects of optical defocus on central (foveal) and peripheral retina')
ax7.boxplot(data)
plt.xticks([1, 2, 3], ['myopic defocus peripheral left', 'centre fovea area', 'hyperopic defocus peripheral right'])
plt.grid(axis='y')
plt.show()

In [ ]:
# OD_pre_nans[OD_pre_nans>10] = 10

display_data = OS_pre_nans
display_data[display_data>10] = 10
fig = go.Figure(data=[go.Surface(z=-display_data)])


fig.update_layout(scene_camera_eye=dict(y=-4, x=0.01, z=-12),
                  scene_aspectmode='manual', 
                  scene_aspectratio=dict(x=12, y=9, z=10),
                  scene = dict(xaxis = dict(nticks=10, range=[0,512],),
                               yaxis = dict(nticks=10, range=[0,256],),
                               zaxis = dict(nticks=10, range=[-30,30],),),)

fig.show()

In [ ]:
max_exclusion = 10
mask = np.ones((256, 512))
idxs = np.where(OS_pre_nans>=max_exclusion)
mask[idxs[0], idxs[1]] = 0
idxs = np.where(OS_post_nans>=max_exclusion)
mask[idxs[0], idxs[1]] = 0
idxs = np.where(mask == 0)

OS_initial_mean = np.nanmean(OS_pre_layers, axis=0)
OS_post_mean = np.nanmean(OS_post_layers, axis=0)

OS_initial_std = np.nanstd(OS_pre_layers, axis=0)
OS_post_std = np.nanstd(OS_post_layers, axis=0)

display_data = (OS_post_mean - OS_initial_mean) 
display_data[idxs] = np.nan

fig = go.Figure(data=[go.Surface(z=np.array(display_data.T))],)

title_name = ('Choroid thickness, '+ scan_type.split('_')[-1] +', STD post intervention' +
              ', avg: ' + str(np.round(np.nanmean(display_data), 2)) +
              ', std: ' + str(np.round(np.nanstd(display_data), 2))  +
              ', max: ' + str(np.round(np.nanmax(display_data), 2)) + 
              ', min: ' + str(np.round(np.nanmin(display_data), 2)) 
             ) 

fig.update_layout(title=title_name, title_font_size=22, autosize=False,
                  width=1200, height=900,
                  margin=dict(l=50, r=50, b=50, t=50), 
                  
                  )
# fig.ColorBar(cmid=0)
fig.update_layout(coloraxis = dict(colorscale='balance', colorbar_len=0.7),)
# #                     colorbar_title_side='right',
# #                     colorscale='RdYlGn',
# #                     colorbar_tickfont=dict(size=18, ),
# #                     colorbar_title_font=dict(size=18, ),
# #                     reversescale=True
#                      )

fig.update_layout(scene_camera_eye=dict(x=4, y=-0.01, z=22),
                  scene_aspectmode='manual', 
                  scene_aspectratio=dict(x=9, y=12, z=10),
                  scene = dict(xaxis = dict(nticks=10, range=[0,256],),
                               yaxis = dict(nticks=10, range=[0,512],),
                               zaxis = dict(nticks=10, range=[-30,30],),),)
fig.update_layout(scene = dict(
                    xaxis_title='vertical pixel position',
                    yaxis_title='thickness map, horizontal pixel position',
                    zaxis_title='thickness in micrometer'),
                 )

fig.show()



In [ ]:
print('mean: ', np.nanmean(display_data), ' std: ', np.nanstd(display_data))
fig_hist =plt.hist(display_data.flatten(), 100)

map_significant_data(display_data, save_file=False, output_fpn=None)

In [ ]:
# calculating p-Values with Wilcoxon stats test
p_values = appl_stats_on_map(OS_pre_layers, OS_post_layers)

In [ ]:
mask = np.ones((256, 512))
idxs = np.where(OS_pre_nans>=max_exclusion)
mask[idxs[0], idxs[1]] = 0
idxs = np.where(OS_post_nans>=max_exclusion)
mask[idxs[0], idxs[1]] = 0
idxs = np.where(mask == 0)


display_data = p_values
display_data[idxs[0], idxs[1]] = np.nan

im_output_fn = 'OS_all_wilcoxon_choroid_thickness_delta_reg_mm_outlierremoval.png'

title_name = ('Choroid thickness mean residual stats, OS, all px, Wilcoxon signed-rank test') 
map_pValues(p_values)


In [ ]:
display_data = (OS_post_mean - OS_initial_mean) 

idxs = np.where(OS_pre_nans>=max_exclusion)
display_data[idxs] = np.nan
idxs = np.where(OS_post_nans>=max_exclusion)
display_data[idxs] = np.nan

idxs = np.where(p_values>0.05)
display_data[idxs] = np.nan
title_name = ('Choroid thickness areas of significant change, OS, stats Wilcoxon signed-rank test') 
map_significant_data(display_data, save_file=False, output_fpn=None)